In [ ]:
import os
import re
import json
import fitz
import spacy
import faiss
import pickle
import win32com.client
from docx import Document
from RAGLibrary import Widgets, ExtractData, SegmentChunks

os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
widgets_list = Widgets.create_name_form()

In [ ]:
""" DEFINE """

""" GET PARENT VALUE """
data   = widgets_list[0] #HBox 1
keys   = widgets_list[1] #HBox 2
choose = widgets_list[2] #HBox 3

embedd_model = widgets_list[3]
search_egine = widgets_list[4]
rerank_model = widgets_list[5]
respon_model = widgets_list[6]
API_drop     = widgets_list[7]

# Hbox 4
chunk_input = widgets_list[8]

# HBox 5
level_value = widgets_list[9]

# Button
button_box  = widgets_list[10]


""" GET CHILDREN VALUE """

# HBox 1
file_name = data.children[0]
file_type = data.children[1]
path_end  = data.children[2]

# HBox 2
data_key = keys.children[0]
embe_key = keys.children[1]

# HBox 3
switch_model = choose.children[0]
merge_otp    = choose.children[1]

# HBox 4
level_input = chunk_input.children[0]
word_limit  = chunk_input.children[1]

# HBox 5
level_values = [child.value for child in level_value.children]

""" DEF VALUE """

# Get value
data_folder   = file_name.value
file_type_val = file_type.value

data_key_val  = data_key.value
embe_key_val  = embe_key.value

API_key_val = API_drop.value
switch      = switch_model.value
merge       = merge_otp.value
path_end_v  = path_end.value

embedding_model = embedd_model.value
searching_egine = search_egine.value
reranking_model = rerank_model.value
responing_model = respon_model.value

level_input_val = level_input.value
word_limit_val  = word_limit.value


""" DETAIL """

# Define
dcmt_path = f"../Doc/{data_folder}{path_end_v}"

base_folder = "../Data"
base_path = f"{base_folder}/{data_folder}/{file_type_val}_{data_folder}"

chunks_base    = f"{base_path}_Chunks.json"
json_file_path = f"{base_path}_Database.json"
schema_ex_path = f"{base_path}_Schema.json"
embedding_path = f"{base_path}_Embeds_{merge}"

torch_path  = f"{embedding_path}.pt"
faiss_path  = f"{embedding_path}.faiss"
mapping_path = f"{embedding_path}_mapping.json"
mapping_data = f"{embedding_path}_map_data.json"

FILE_TYPE    = file_type_val
DATA_KEY     = data_key_val
EMBE_KEY     = embe_key_val
SWITCH       = switch
EMBEDD_MODEL = embedding_model
SEARCH_EGINE = searching_egine
RERANK_MODEL = reranking_model
RESPON_MODEL = responing_model

if FILE_TYPE == "Data":
    MERGE = merge
else: 
    MERGE = "no_Merge"

API_KEY = API_key_val

SEARCH_ENGINE = faiss.IndexFlatIP

print("\n")
print(f"Embedder: {EMBEDD_MODEL}")
print(f"Searcher: {SEARCH_EGINE}")
print(f"Reranker: {RERANK_MODEL}")
print(f"Responer: {RESPON_MODEL}")
print(f"Data Key: {DATA_KEY}")
print(f"Embe Key: {EMBE_KEY}")
print(f"Dcment  : {dcmt_path}")
print(f"Chunked : {chunks_base}")
print(f"Database: {json_file_path}")
print(f"Torch   : {torch_path}")
print(f"Faiss   : {faiss_path}")
print(f"Mapping : {mapping_path}")
print(f"Map Data: {mapping_data}")
print(f"Schema  : {schema_ex_path}")
print(f"Model   : {SWITCH}")
print(f"Merge   : {MERGE}")
print(f"API Key : {API_KEY}")
print(f"Word    : {word_limit_val}")
print(f"Level   : {level_input_val}")
print(f"Level Values: {level_values}")

In [ ]:
path = dcmt_path
Contents = level_values[-1] if level_values else None
print(Contents)

BASE


In [ ]:
# ADD CHUNKS
def add_chunk(chunks, content):
    if content["Chương"] and content[Contents]:
        content["Index"] += 1
        chunks.append(content.copy())
        content[Contents] = []
        
def is_chapter(text):
    text = text.strip()
    return bool(re.match(r"^Chương\s*[IVXLCDM\d]+\b", text, re.IGNORECASE))

def is_article(text):
    text = text.strip()
    return bool(re.match(r"^Điều\s+([IVXLCDM\d]+)\.\s*(.+)", text, re.IGNORECASE))

def is_clause(text):
    text = text.strip()
    return bool(re.match(r"^\d+\.\s+.+", text))

def is_content(text):
    text = text.strip()
    return bool(re.match(r'^([-+*•●◦○] )|([a-zA-Z\-\+\*]+[.)\]:] )|(\(\w+\) )', text))


In [ ]:
# MAIN FUNCTION
def main(text_data):
    chunks = []
    content = {"Index": 0, "Chương": None, "Điều": None, "Khoản": None, Contents: []}
    i = 0
    while i < len(text_data):
        chunk = text_data[i]["text"]

        if is_chapter(chunk):
            if i + 1 < len(text_data):
                chunk += f": {text_data[i + 1]['text']}"
            add_chunk(chunks, content)
            content["Chương"] = chunk
            content["Điều"] = None
            content["Khoản"] = None 
            i += 1

        elif is_article(chunk):
            match = re.match(r"^(Điều\s*[IVXLCDM\d]+)\.\s*(.+)", chunk, re.IGNORECASE)
            if content["Chương"]:
                if match:
                    chunk = f"{match.group(1)}: {match.group(2)}"
                add_chunk(chunks, content)
                content["Điều"] = chunk
                content["Khoản"] = None 
        
        elif is_clause(chunk):
            match = re.match(r"^(\d+)\.\s*(.+)", chunk)
            if content["Chương"]:
                if match:
                    clause_number = match.group(1)
                    clause_content = match.group(2)

                    if i + 1 < len(text_data) and is_content(text_data[i + 1]["text"]):
                        chunk = f"Khoản {clause_number}: {clause_content}"
                        add_chunk(chunks, content)
                        content["Khoản"] = chunk
                    else:
                        chunk = f"Khoản {clause_number}"
                        add_chunk(chunks, content)
                        content["Khoản"] = chunk

                        chunk = clause_content
                        content[Contents].append(chunk)
                else: 
                    print(chunk)

        elif is_content(chunk):
            match = re.match(r'^([-+*•●◦○a-zA-Z\-\+\*]+[.)\]:] )(\s.+)', chunk)
            if content["Chương"]:
                if match:
                    chunk = match.group(2)
                content[Contents].append(chunk)
        i += 1
        
    return chunks

In [11]:
# CHUNKS BASE
text_data = ExtractData.extracted(dcmt_path)
chunks = main(text_data)

In [ ]:
# EXPORT BASE
with open(chunks_base, "w", encoding="utf-8") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)

print(f"Base data saved to {chunks_base}")

FINAL


In [ ]:
nlp = spacy.load("en_core_web_sm")
WORD_LIMIT = int(word_input.value) if word_input.value.isdigit() else 200

In [ ]:
SegmentChunks.process_json(chunks_base, json_file_path, Contents, word_limit, markers, nlp)